# Эксперименты с series доменом

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

In [77]:
from pathlib import Path
from tqdm import tqdm
import pickle
from time import time
from tqdm import tqdm
import pandas as pd
import numpy as np

import mlflow
import optuna

from src.models.BaseModel import TopRecommender
from src.models.ItemBasedRecommenders import CosineDistanceRecommender
from src.models.MatrixFactorizationRecommenders import ALSBasedRecommender, LightFMBasedRecommender

from src.metrics import average_single_precision

In [3]:
# Настраиваем MLFlow
mlflow.set_tracking_uri((Path.cwd() / '../src/models/mlflow_tracking/mlruns'))
mlflow.set_experiment('Series Domain')

2023/04/15 19:10:23 INFO mlflow.tracking.fluent: Experiment with name 'Series Domain' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/trybi/PycharmProjects/MFDP-RecSys/src/models/mlflow_tracking/mlruns/2', creation_time=1681575023415, experiment_id='2', last_update_time=1681575023415, lifecycle_stage='active', name='Series Domain', tags={}>

In [4]:
data_path = Path('../data')

In [6]:
df_interactions = pd.read_parquet(data_path / 'series_interactions.parquet')
df_series = pd.read_parquet(data_path / 'series_bd.parquet')

In [8]:
df_interactions = df_interactions.reset_index().drop('index', axis=1)

In [9]:
df_interactions

,user_id,item_id,rating
0,864613,s7638,1
1,15607,s3043,1
2,81786,s2616,1
3,474544,s10440,1
4,810825,s4223,1
...,...,...,...
447401,13243,s2657,-1
447402,565181,s15297,-1
447403,616080,s15297,-1
447404,337043,s9689,1


In [10]:
# Отложим 1000 наблюдений в качестве тестовой выборки. На них будем считать метрику MAP@10.
# Немного наблюдений в качестве теста, потому что кажется, что дело это не быстрое. 

test_samples = np.random.choice(range(len(df_interactions)), size = 1000, replace=False)

df_train = df_interactions.drop(test_samples)
df_test = df_interactions.iloc[test_samples]

In [11]:
# Соберем словарь {user: [items]}, потому что так быстрее будет работать поиск истории по юзерам
train_db = {user: items.item().split(', ') for user, items in (df_train
                                                               .groupby('user_id')
                                                               .agg({'item_id': lambda x: ', '.join(x)})
                                                               .iterrows())}

In [12]:
# Посмотрим максимальное количество посмотренных сериалов. Спойлер - не так много
max(len(i) for i in train_db.values())

70

In [13]:
# Соберем список пар (user: item) из test, чтобы по нему итерироваться
test_data = df_test[['user_id', 'item_id']].values

In [31]:
# Функция, которая будет считать эксперименты 
def experiment(run_name, model, model_params, recommend_params, my_favorites, k=10):
    with mlflow.start_run(run_name=run_name):
        mlflow.log_params(model_params)
        
        time_start = time()
        model.fit(interaction_data=df_train.rename({'user_id': 'user', 'item_id': 'item'}, axis=1),
                  **model_params)
        mlflow.log_metric(f'fit_time', time()-time_start)
        
        maps = []
        times = []
        for new_user, new_item in tqdm(test_data):
            user_favorites = train_db.get(new_user, [])
            time_start = time()
            recs = model.recommend(user_favorites, k, **recommend_params)
            times.append(time()-time_start)
            maps.append(average_single_precision(new_item, recs))
        
        
        mlflow.log_metric(f'MAP_on_{k}', sum(maps)/len(maps))
        mlflow.log_metric(f'time_to_rec', sum(times)/len(times))
        
        my_recs = model.recommend(my_favorites, k, **recommend_params)
        
        
        mlflow.log_text(' \n '.join(i + ' - ' + j + ' - ' + z for k in my_recs for i, j, z in df_series.query('item_id==@k')[['title', 
                                                                                                                              'genres',
                                                                                                                              'countries']
                                                                                                                            ].values),
                        'my_recs.txt')

        with open(model_path / f'{run_name}_model_series.pickle', 'wb') as f:
            pickle.dump(model, f)

        mlflow.log_artifact(model_path / f'{run_name}_model_series.pickle')

In [35]:
model_path = Path('../src/models/models_storage')

In [36]:
my_favorites = ['s9157', 's15314', 's1204']

In [37]:
df_series[df_series.item_id.isin(my_favorites)]

,item_id,title,release_year,genres,countries,age_rating,directors
4890,s1204,Почему женщины убивают,2019.0,"драмы, мелодрамы, триллеры, комедии",США,18.0,"Дэвид Гроссман, Дэвид Уоррен"
9720,s9157,Твин Пикс,1990.0,"драмы, мистика, триллеры, детективы",США,18.0,"Дэвид Линч, Калеб Дешанель, Лесли Линка Глаттер"
11848,s15314,Острые козырьки,2013.0,"драмы, криминал",Великобритания,16.0,"Колм МакКарти, Тим Милантс, Дэвид Кэффри"


In [39]:
experiment('Top count recommends', TopRecommender(), {'n_to_save':100, 'metric':'count'}, {'mode':'deterministic'},
           my_favorites, k=10)

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 5649.61it/s]


In [40]:
experiment('Random recommends', TopRecommender(), {'n_to_save':1000, 'metric':'count'}, {'mode':'probabilistic'},
           my_favorites, k=10)

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1282.05it/s]


In [41]:
experiment('CosineDistanceRecommender', CosineDistanceRecommender(), {}, {'mode':'deterministic'},
           my_favorites, k=10)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 732.48it/s]


In [59]:
experiment('CosineDistanceRecommender + random', CosineDistanceRecommender(), {}, {'mode':'probabilistic'},
           my_favorites, k=10)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 580.60it/s]


In [72]:
experiment('ALS Recommender', ALSBasedRecommender(), {}, {},
           my_favorites, k=10)

  0%|          | 0/15 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 745.16it/s]


In [73]:
df_train['rating'] += 2

In [74]:
experiment('ALS Recommender + changed DS', ALSBasedRecommender(), {}, {},
           my_favorites, k=10)

  0%|          | 0/15 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 734.75it/s]


In [75]:
experiment('ALS Recommender + changed DS', ALSBasedRecommender(), {'factors': 10, 'regularization': 0.5, 'iterations': 30}, {},
           my_favorites, k=10)

  0%|          | 0/30 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1026.70it/s]


In [76]:
experiment('ALS Recommender + changed DS', ALSBasedRecommender(), {'factors': 50, 'regularization': 1, 'iterations': 30}, {},
           my_favorites, k=10)

  0%|          | 0/30 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 840.33it/s]


## Попробуем подобрать гипперы для ALS

In [78]:
# Отложим 1000 наблюдений в качестве валидационной выборки

val_samples = np.random.choice(range(len(df_train)), size = 1000, replace=False)

df_train_val = df_train.iloc[val_samples]
df_train_train = df_train[~df_train.index.isin(df_train_val.index)]

In [85]:
train_db_new = {user: items.item().split(', ') for user, items in (df_train_train
                                                               .groupby('user_id')
                                                               .agg({'item_id': lambda x: ', '.join(x)})
                                                               .iterrows())}

In [89]:
# Напишем функцию для подбора гипперов
def objective(trial):
    factors = trial.suggest_int("factors", 5, 100)
    regularization = trial.suggest_float("regularization", 0, 10)
    
    model.fit(interaction_data=df_train_train.rename({'user_id': 'user', 'item_id': 'item'}, axis=1),
              factors = factors,
              regularization = regularization,
              iterations=30)
    
    maps = []
    for new_user, new_item in df_train_val[['user_id', 'item_id']].values:
        user_favorites = train_db_new.get(new_user, [])
        recs = model.recommend(user_favorites, 10)
        maps.append(average_single_precision(new_item, recs))

    
    return sum(maps)/len(maps)

In [90]:
model = ALSBasedRecommender()

In [92]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2023-04-18 01:15:55,827] A new study created in memory with name: no-name-79b976d2-d68c-4c37-864d-6406a640a1e2


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2023-04-18 01:16:13,914] Trial 0 finished with value: 0.016224960317460315 and parameters: {'factors': 38, 'regularization': 9.919748184947982}. Best is trial 0 with value: 0.016224960317460315.


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2023-04-18 01:17:12,526] Trial 1 finished with value: 0.019889325396825423 and parameters: {'factors': 97, 'regularization': 7.748745346339712}. Best is trial 1 with value: 0.019889325396825423.


...

[I 2023-04-18 01:40:48,340] Trial 47 finished with value: 0.020122777777777812 and parameters: {'factors': 89, 'regularization': 8.09797135992447}. Best is trial 42 with value: 0.02103257936507941.


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2023-04-18 01:41:25,304] Trial 48 finished with value: 0.019988452380952416 and parameters: {'factors': 86, 'regularization': 8.560537711175737}. Best is trial 42 with value: 0.02103257936507941.


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2023-04-18 01:41:54,654] Trial 49 finished with value: 0.02069789682539687 and parameters: {'factors': 81, 'regularization': 9.538592831061518}. Best is trial 42 with value: 0.02103257936507941.


In [95]:
study.best_params

{'factors': 97, 'regularization': 9.418154957993593}

In [96]:
experiment('ALS Recommender + changed DS + optuna', ALSBasedRecommender(), {'factors': 97,
                                                                            'regularization': 9.418154957993593,
                                                                            'iterations':30}, {},
           my_favorites, k=10)

  0%|          | 0/30 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 569.15it/s]
